In [11]:
import matplotlib.pyplot as plt
import numpy as np
import random as ran
import tensorflow as tf


In [12]:
# Loading data
# We have used "One-hot encoding" -- in which a given label is converted from single number to
# vector whose length equals the number of possible classes.

# For eg: If given label is of an image charater "7" -- It's corresponding vector would be having 
# 10 elements, with all elements be 0 and only 7th element be 1.
# Ref: https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
# Keep the test lables (classes) as single numbers as well for various comparison and performance measures.
# So we convert the One-Hot encoded vectors back to a single number by taking the index of the highest element. 

mnist.test.cls = np.array([label.argmax() for label in mnist.test.labels])

In [14]:
# Data dimensions
img_size = 28                         # Image size -- 28 X 28 pixels (in MNIST data set).
img_size_flat = img_size * img_size   # Array length  -- 28 X 28 = 784 -- Images are flatten 
                                      # and then stored in 1D array of size 784.
img_shape = (img_size, img_size)      # Tuple with height and width of images used to reshape arrays.
num_classes = 10                      # Number of classes, one class for each of 10 digits.


In [15]:
# Computational Graph using TensorFlow

x = tf.placeholder(tf.float32, [None, img_size_flat])
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])

weights = tf.Variable(tf.zeros([img_size_flat, num_classes]))
biases = tf.Variable(tf.zeros([num_classes]))

In [16]:
# Model

logits = tf.matmul(x, weights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)

In [17]:
# Cost Function

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [18]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [19]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
# TensorFLow Seesion

session = tf.Session()
session.run(tf.global_variables_initializer())
batch_size = 100


def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch = mnist.train.next_batch(batch_size)
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

In [21]:
feed_dict_test = {x: mnist.test.images,
                  y_true: mnist.test.labels,
                  y_true_cls: mnist.test.cls}


In [22]:
# function to print accuracy
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [23]:
# Check the accuracy
print_accuracy()

Accuracy on test-set: 9.8%


In [24]:

optimize(num_iterations=1000)
print_accuracy()

Accuracy on test-set: 91.0%
